In [1]:
!pip install selenium
!pip install selenium-wire
!pip install beautifulsoup4
!pip install lxml

In [2]:
#scraper class to initialize standard features

import json
import base64
from bs4 import BeautifulSoup

from urllib import request
from urllib.parse import urlsplit
from urllib.parse import urlparse
import urllib.parse
import socket

import os
import inspect

import uuid #used to generate random file names

import time

class Scraping:

    def __init__(self):
        self = self

    def __del__(self):
        print('Helper object destroyed')

    def encode_code(self, code):
        code = code.encode('utf-8','ignore')
        code = base64.b64encode(code)
        return code

    def decode_code(self, value):


        try:
            code_decoded = base64.b64decode(value)
            code_decoded = BeautifulSoup(code_decoded, "html.parser")
            code_decoded = str(code_decoded)
        except Exception as e:
            print(str(e))
            code_decoded = "decoding error"
        return code_decoded



    def decode_picture(self, value):
        picture = value.tobytes()
        picture = picture.decode('ascii')
        return picture

    def get_result_meta(self, url):
        meta = {}
        ip = "-1"
        main = url
        #parse url to get hostname and socket
        try:
            parsed_uri = urlparse(url)
            hostname = '{uri.netloc}'.format(uri=parsed_uri)
            ip = socket.gethostbyname(hostname)
        except Exception as e:
            print(str(e))
            ip = "-1"

        try:
            main = '{0.scheme}://{0.netloc}/'.format(urlsplit(url))
        except Exception as e:
            print(str(e))
            main = url

        #write to meta dictionary
        meta = {"ip":ip, "main":main}

        return meta

    def get_chrome_extension(self):
        #change extension path to load the I-don't care about cookies file
        
        currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
        parentdir = os.path.dirname(currentdir)
        
        #check for the used operating system to load the correct folder structure
        if os.name == "nt":
            extension_path = r'C:\Users\Hilfskraft\Documents\Scraper\I-don-t-care-about-cookies.crx'
        else:
            extension_path = '//I-don-t-care-about-cookies.crx'
        return extension_path


    def take_screenshot(self, driver):

        #function to encode file content to base64
        def encode_file_base64(self, file):
            f = open(file, 'rb')
            code = f.read()
            code = base64.b64encode(code)
            f.close()
            return code

        current_path = os.path.abspath(os.getcwd())

        #iniatilize constant variables

        #iniatilize the directories for the extension and for the folder for temporary downlods of files
        
        #check for the used operating system to load the correct folder structure
        if os.name == "nt":
            screenshot_folder = r'C:\Users\Hilfskraft\Documents\Scraper\tmp'


        else:
            screenshot_folder = "//tmp//"

        screenshot_file = screenshot_folder+str(uuid.uuid1())+".png"
              
        time.sleep(2)

        driver.maximize_window() #maximize browser window for screenshot
        
        try:
            driver.execute_script("window.scrollTo(0,1)")
        except Exception as e:
            print(str(e))
            pass

        #try to get the whole browser window
        try: 
            required_width = driver.execute_script('return document.body.parentNode.scrollWidth')
            required_height = driver.execute_script('return document.body.parentNode.scrollHeight')
                      
            scroll = "window.scrollTo(0,{})".format(required_height)
            
            driver.execute_script(scroll)

            required_height+= 50
            
            driver.execute_script("window.scrollTo(0,1)")
            
            driver.set_window_size(required_width, required_height)
            
            driver.maximize_window()
            
            driver.save_screenshot(screenshot_file) #take screenshot
            
        except Exception as e:
            print(str(e)) ##next try based on the document.body

            try: 
                body_screenshot = driver.find_element(By.TAG_NAME, "body")
                body_screenshot.screenshot(screenshot_file)
            except Exception as e:
                print(str(e)) #if all fails take screenshot of the browser view
                driver.save_screenshot(screenshot_file) #take screenshot

        #open screenshot and save as base64
        screenshot = encode_file_base64(self, screenshot_file)

        os.remove(screenshot_file) #remove the comment if you don't want to store the screenshots

        return screenshot #return base64 code of image

    #function to read out redirected urls

    def get_real_url(url, driver):
                
        try:
            driver.get(url)
            time.sleep(4)
            current_url = driver.current_url #read real url (redirected url)
            driver.quit()
            return current_url
        except Exception as e:
            print(str(e))
            pass
        


In [3]:
"""
This template describes the steps necessary to add a custom scraper for the RAT software. First of all, it is assumed that these are search services that provide search forms. However, it is also possible to add other search systems. For this, the procedure would have to be adapted accordingly. Selenium is used as the basis for scraping.

For scraping search results it is necessary to develop the name of the search service to be scraped as well as a scraper Python file for it. In this file appropriate functions must be defined, which are identical for all search engines. However, the contents can also be designed very individually. It is important that at the end search results with the following contents are returned, whereby fields can be filled also empty or with placeholders:

- result_title: title in the snippet of a result
- result_description: description in the snippet of a result
- result_url: url of the search result
- serp_code: html source of the search result page, if it is used for other analyses
- serp_bin: screenshot of the search result page, if it is needed for other analyses
- page: most search engines offer the possibility to browse the result pages. This variable is needed to scroll further. However, for search services where new search results are added by scrolling, it must be adjusted accordingly.

Overall, a scraper usually consists of the following functions:
- run(query, limit, scraping): main function for all scrapers with the following parameters: query = search query, limit = maximum number of results to query, scraping = scraping object with functions to scrape the search engines
- get_search_results(driver, page): subfunction to retrieve the search results with the following parameters:
 driver = selenium driver; webbrowser for scraping; page = SERP page
- check_captcha(driver): Helper function to check if there is a lockout of the search services. The function is called to indicate that scraping has been aborted. If checking it is False, the scraping will continue.

The following will describe all standard variables and functionality to scrape a search engine. But again, it is also possible to change it all accordingly to the search engine to be scraped.

Link to the documentation on locating elements in Selenium: https://selenium-python.readthedocs.io/locating-elements.html

"""

#required libs for web scraping

#import external libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.common.exceptions import TimeoutException #used to interrupt loding of websites and needed as workaround to download files with selenium
from selenium.webdriver.common.action_chains import ActionChains #used to simulate pressing of a key

import uuid #used to generate random file names

import time #used to do timeout breaks

import os #used for file management

#base64 encoding to convert the code codes of webpages
import base64

#BeautifulSoup is necessary to beautify the code coded after it has been decoded (especially useful to prevent character errors)
from bs4 import BeautifulSoup
from lxml import html

import random

#main function to run a scraper

def run(query, limit, scraping, headless):
    try:
        #Definition of args for scraping the search engine
        search_url = "https://search.brave.com/" #URL of search engine, e. g. www.google.de
        search_box = "q" #Class name of search box; input field for searches
        captcha = "g-recaptcha" #Source code hint for CAPTCHA; some search engines use CAPTCHAS to block too many automatic requests
        next_page = "//a[@class='btn svelte-o26zbb']" #CSS to find click on next SERP; for search engines that use a navigation on SERPS to continue browsing more search results
        results_number = 0 #initialize results_number; normally starts at 0
        page = 1 #initialize SERP page; normally starts at 1
        search_results = [] #initialize search_results list

        #Definition of custom functions

        #Function to scrape search results
        def get_search_results(driver, page):
            
            counter = 0

            get_search_results = [] #temporary list to store search results

            source = driver.page_source #storing the source code of a search engine result page

            serp_code = scraping.encode_code(source) #use the function from the lib scraping to encode the source code to base64; preparing it to be stored in database

            serp_bin = scraping.take_screenshot(driver) #use the function to take a screenshot, encoding it to base64, to store it in database

            soup = BeautifulSoup(source, features="lxml") #use BeautifulSoup to read the source code of search engine result page and prepare it to read the tree to extract the search results

            #Procedure to read out the search results with title, description and URL. There are several ways to extract such information. In this example we will use functions from BeautifulSoup to read out content of Div containers. Other approaches could be using Selenium or XPATH. Choosing the best way always depends on the search engine.
            
            #first find each block with one result
            
            for result in soup.find_all("div", class_=["snippet svelte-uqu4ua"]):
                
                url_list = []

                result_description = "N/A"
                result_url = "N/A"
                search_result = []
                result_title = ""
                result_description = ""
                
                #try to extract the title based on a css class for the link title
                try:
                    for title in result.find("div", class_=["title svelte-1mekd5e"]):
                        result_title=title.text.strip()
                except:
                    result_title = "N/A"

                try:
                    #try to extract the result description by css class and change the content of the description, if necessary:

                    for description in result.find_all("div", class_=["snippet-description"]):
                        result_description+=description.text.strip()
                        result_description = result_description.replace(result_title, " ")
                        #result_description = result_description.replace("\n", "")
                        result_description = " ".join(result_description.split())
                except Exception as e:
                    result_description = "N/A"
                    
                #try to extract the urls of the seaech results and change the url if the search engines works with relative hyperlinks
                try:
                    for url in result.find_all("a"):
                        url = url.attrs['href']
                        url_list.append(url)
                        result_url = url_list[0]
                except:
                    result_url = "N/A"
                    
                if counter < limit+2:
                    
                    get_search_results.append([result_title, result_description, result_url, serp_code, serp_bin, page]) #call function to read search results and append the results to the list
                    counter += 1
                    
            return get_search_results

        #Function to check if search engine shows CAPTCHA code. some search engines notice that a bot is trying to scrape their results. if that happens they tend to show a message or captcha
        def check_captcha(driver):
            source = driver.page_source
            if captcha in source:
                return True
            else:
                return False

        chrome_extension = scraping.get_chrome_extension() #Get Path for I don't care about cookies extension

        #initialize Selenium with options and arguments. Chrome supports a lot of arguments, a comprehensive list can be found here: https://peter.sh/experiments/chromium-command-line-switches/
        options = Options()
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        if headless == 1: #argument to check if browser should be started in headless mode or not. 1 = without browser head; 0 = with browser head for debuggin
            options.add_argument('--headless=new')
        options.add_argument("--start-maximized")
        options.add_experimental_option("detach", True)
        options.add_argument("--lang=de")
        options.add_extension(chrome_extension)
        
        driver = webdriver.Chrome(options=options)
        driver.set_page_load_timeout(20)
        driver.implicitly_wait(30)
        driver.get(search_url)
        random_sleep = random.randint(2, 5) #random timer trying to prevent quick automatic blocking
        time.sleep(random_sleep)
        

        #Start scraping if no CAPTCHA; not necessary if search engine doesn't use mechanism to block automatic requests
        if not check_captcha(driver):


            #commands to trigger a search; some search engines just use URL GET-pararmeters which could be passed too
            search = driver.find_element(By.NAME, search_box)
            search.send_keys(query)
            search.send_keys(Keys.RETURN)

            search_results = get_search_results(driver, page)
            results_number = len(search_results)

            if headless == 1:
                driver.quit()
            return search_results


        else:
            search_results = -1
            if headless == 1:
                driver.quit()
            return search_results

    except Exception as e:
        print(str(e))
        try:
            driver.quit()
        except:
            pass
        search_results = -1
        return search_results

In [4]:
#function to test the scraper. it shows all scraped details or an error message, if it fails

def test_scraper(query, limit, scraper, headless):
    search_results = run(query, limit, scraper, headless)
    
    i = 0
    if search_results != -1:
        for sr in search_results:
            i+=1
            print(i)
            print(sr[0])
            print(sr[1])
            print(sr[2]) 
    else:
        print("Scraping failed")

In [5]:
#initialize the scraper
scraper = Scraping() #initialize the scraping object
query = "manga" #search query
limit = 7 #max_number of results (the scraper normally adds some more pages since not all search engines deliver a certain number of search results on every SERP)
headless = 0 #0 = don't start selenium in headless mode (good for debugging); 1 = start selenium in headless mode

In [6]:
test_scraper(query, limit, scraper, headless)

1
Manga – Wikipedia
12. August 2002 - Manga [ˈmaŋɡa] (japanisch 漫画) ist der japanische Begriff für Comics. Außerhalb von Japan bezeichnet er meist ausschließlich aus Japan stammende Comics, wird aber auch für nichtjapanische Werke verwendet, die visuell und erzählerisch stark an japanische Vorbilder angelehnt sind.
https://de.wikipedia.org/wiki/Manga
2
Manga Bücher online kaufen | Thalia
26. April 2017 - Hier im Thalia-Onlineshop erwartet Sie ein vielfältiges Angebot an Mangas von Tokyopop, Egmont Manga und anderen bekannten Verlagen. Sie finden diverse humorvolle Mangas ebenso wie ernsthafte oder actionreiche. Die Bezeichnung Manga bezieht sich auf Comics, die aus Japan stammen.
https://www.thalia.de/kategorie/manga-94/
3
Manga von Carlsen Manga! und Hayabusa | Carlsen
31. Januar 2023 - Entdecke die Titel von Carlsen Manga!, darunter Bestseller wie NARUTO, MY HERO ACADEMIA, ATTACK ON TITAN , DRAGON BALL, ONE PIECE, FRUITS BASKET oder FAIRY TAIL.
https://www.carlsen.de/manga
4
Lies Bel